In [30]:
# This is my Voice Assistant

In [31]:
from vosk import Model, KaldiRecognizer
import pyaudio
#windows use pipwin
import pyttsx3
from datetime import datetime
import geocoder
import requests
import json
from dotenv import load_dotenv
import os

In [32]:
load_dotenv()
#pulled from env
# Weather - Tomorrow.io - Api Key
weather_key = os.environ.get("weather_tmr_api")
# Vosk Model Path
model_path = os.environ.get("vosk_small_eng_path")

In [33]:
#https://alphacephei.com/vosk/models
#Download english model and find the file path
model = Model(model_path)
recognizer = KaldiRecognizer(model, 16000)

In [34]:
mic = pyaudio.PyAudio()
listening = False
speaker = pyttsx3.init()
speaker.setProperty('volume', 1.5)

In [35]:
# weather Code Dict
weatherCode = {
    "0": "Unknown",
    "1000": "Clear or Sunny skies",
    "1100": "Mostly Clear skies",
    "1101": "Partly Cloudy skies",
    "1102": "Mostly Cloudy skies",
    "1001": "Cloudy skies",
    "2000": "some Fog",
    "2100": "Light Fog",
    "4000": "a light Drizzle",
    "4001": "Rain",
    "4200": "Light Rain",
    "4201": "Heavy Rain",
    "5000": "Snow",
    "5001": "Flurries",
    "5100": "Light Snow",
    "5101": "Heavy Snow",
    "6000": "Freezing Drizzle",
    "6001": "Freezing Rain",
    "6200": "Light Freezing Rain",
    "6201": "Heavy Freezing Rain",
    "7000": "Ice Pellets",
    "7101": "Heavy Ice Pellets",
    "7102": "Light Ice Pellets",
    "8000": "Thunderstorms"
}

In [36]:
def get_command():
    listening = True
    stream = mic.open(format = pyaudio.paInt16, channels = 1, rate = 16000, input = True, frames_per_buffer = 8192)
    while(listening):
        stream.start_stream()
        try:
            data = stream.read(4096)
            if recognizer.AcceptWaveform(data):
                speech = recognizer.Result()
                text = speech[14:-3]
                listening = False
                stream.close()
                return text
        except OSError:
            pass

In [37]:
def execute_command(command):
    try:
        print(command)
        if "what's the weather" in command or "today's weather" in command or "today weather" in command:
            weather_today()
        if "what's the time" in command or "what time is it" in command:
            time_now()
        # if "today's schedule" in command or "schedule today" in command or "today schedule" in command:
            # calendar_today()
    except Exception:
        pass

In [38]:
def speak(text):
    speaker.say(text)
    speaker.runAndWait()
    speaker.stop()

In [39]:
def weather_help():
    # Using API from tomorrow.io
    try:
        my_loc = geocoder.ip('me')
        location = str(my_loc.lat) + '%2C%20' + str(my_loc.lng)
        # URL Builder -> Api refereces above
        url = 'https://api.tomorrow.io/v4/timelines?location='+ location + '&fields=temperature&fields=temperatureApparent&fields=humidity&fields=weatherCode&timesteps=current&units=metric&apikey=' + weather_key
        response_API = requests.get(url)
        data = response_API.text
        if '{"code":429001,"message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume.","type":429}' in data:
            text = 'you have reached the API call limit for today'
            return text
        values = json.loads(data)
        values = values['data']['timelines'][0]['intervals'][0]['values']
        code = str(values['weatherCode'])
        text = "Todays temperature is " + str(values['temperature']) +' celcius. Expect ' + weatherCode.get(code)
        return text
    except Exception:
        pass

In [40]:
# what's the weather || weather today
def weather_today():
    text = weather_help()
    speak(text)

In [41]:
# what's the time || what time is it
def time_now():
    time = datetime.now()
    curr_time = time.strftime("%H:%M")
    text = "It is currently " + curr_time
    speak(text)

In [42]:
# today's schedule || schedule today || today schedule
# def calendar_today():

In [43]:
while True:
    command = get_command()
    if "stop stop" in command:
        text = 'voice assistant offline'
        print(text)
        speak(text)
        break
    execute_command(command)


stop
voice assistant offline
